## Separating Frontscan and Backscan for different number of Sensors

The key ideas here are:

- Functions like moduleAnalysis() returns two identically structured dictionaries that contain the keys like xstart, ystart, zstart, xinc, yinc, zinc, Nx, Ny, Nz, orientation. For the function arguments like sensorsy or sensorsx, there is an assumption that those will be equal for both the front and back surface.

- We need to develop a separate function, pretty much functionally parallel with moduleAnalysis() to bring out the frontscan and backscan separately.....may be two distinct functions with distinct arguments for frontscan() and backscan()

- The new functions will have variables passed on as arguments which can be different for front and back

In [1]:
import bifacial_radiance
import numpy as np
import os
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP')

In [2]:
demo = bifacial_radiance.RadianceObj('ScanSeparate', testfolder) 

path = C:\Users\sarefeen\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP


In [3]:
x = 2
y = 1
xgap = 0.02
ygap = 0.15
zgap = 0.10
numpanels = 1
offsetfromaxis = True
Ny = numpanels
axisofrotationTorqueTube = True
frameParams = None
omegaParams = None
diam = 0.1

In [4]:
module_type = 'TEST'
nMods = 3
nRows = 2
sceneDict = {'tilt':0, 'pitch':6, 'clearance_height':3,'azimuth':90, 'nMods': nMods, 'nRows': nRows} 

In [5]:
demo.setGround(0.2)
epwfile = demo.getEPW(lat = 37.5, lon = -77.6)
metdata = demo.readWeatherFile(epwfile, coerce_year = 2021)
demo.gendaylit(4020)

Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw
 ... OK!
start: 2021-01-01 01:00:00
end: 2021-12-31 23:00:00
Saving file EPWs\epw_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


'skies\\sky2_37.5_-77.33_2021_06_17_13_00.rad'

In [6]:
demo.makeModule(name=module_type,x=x, y=y, torquetube = True, 
                    diameter = diam, xgap = xgap, ygap = ygap, zgap = zgap, 
                    numpanels = Ny, omegaParams=None,
                    axisofrotationTorqueTube=axisofrotationTorqueTube)


Module Name: TEST
Pre-existing .rad file objects\TEST.rad will be overwritten
Module TEST updated in module.json


{'x': 2,
 'y': 1,
 'z': 0.02,
 'modulematerial': 'black',
 'scenex': 2.02,
 'sceney': 1.0,
 'scenez': 0.15,
 'numpanels': 1,
 'bifi': 1,
 'text': '! genbox black TEST 2 1 0.02 | xform -t -1.0 -0.5 0.15 -a 1 -t 0 1.15 0\r\n! genrev Metal_Grey tube1 t*2.02 0.05 32 | xform -ry 90 -t -1.01 0 0',
 'modulefile': 'objects\\TEST.rad',
 'offsetfromaxis': 0.15,
 'xgap': 0.02,
 'ygap': 0.15,
 'zgap': 0.1,
 'cellModule': None,
 'torquetube': {'bool': True,
  'diameter': 0.1,
  'tubetype': 'Round',
  'material': 'Metal_Grey'}}

In [7]:
scene = demo.makeScene(module_type,sceneDict)
octfile = demo.makeOct()
analysis = bifacial_radiance.AnalysisObj()  # return an analysis object including the scan dimensions for back irradiance

Created ScanSeparate.oct


In [8]:
name = 'ModuleScanTest'
rowWanted = 1
modWanted = 2
sensorsy = 2
sensorsx = 3

In [9]:
sensors_diff = True
sensorsx_back = 3
sensorsy_back = 4

In [10]:
frontscan, backscan, start_shift, flag_s = analysis.moduleAnalysis(scene, modWanted=None, rowWanted=None,
                       sensorsy=sensorsy, sensorsx=sensorsx, frontsurfaceoffset=0.001, backsurfaceoffset=0.001, 
                       modscanfront=None, modscanback=None, debug=False, sensors_diff = sensors_diff, 
                       sensorsy_back=sensorsy_back, sensorsx_back=sensorsx_back)

In [11]:
start_shift

-0.5

In [12]:
flag_s

0

In [ ]:
backscan

In [13]:
frontDict, backDict = analysis.analysis(octfile = octfile, name = name, frontscan = frontscan, backscan = backscan, start_shift = start_shift, flag_s = flag_s)

Linescan in process: ModuleScanTest_Front
Linescan in process: ModuleScanTest_Back
Saved: results\irr_ModuleScanTest_Front.csv
Saved: results\irr_ModuleScanTest_Back.csv
